In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from yt.utilities.cosmology import Cosmology
import yt
import yt.units as u
from scipy.integrate import solve_ivp
from scipy.interpolate import InterpolatedUnivariateSpline
from numpy.random import RandomState

In [ ]:
prng = RandomState(24)

In [ ]:
# Planck 2018 cosmology
cosmo = Cosmology(hubble_constant=0.67, omega_matter=0.32, omega_lambda=0.68)
rho_crit = cosmo.critical_density(0.0).to_value("Msun/kpc**3")
h = cosmo.hubble_constant.v
H0 = cosmo.hubble_constant

In [ ]:
K = (6.0*u.G/(u.clight*H0)).in_base("galactic").v

In [ ]:
num_clusters = 1000

In [ ]:
M, numM = np.loadtxt("halo_mass_function.txt", unpack=True, usecols=(0, 8))
M /= h
numM *= h**3

In [ ]:
Q = numM / numM.max()
Q = np.insert(Q, numM.size, 0.0)
P = 1.0-Q
M = np.insert(M, 0, 0.0)

In [ ]:
Pa = np.interp(1.0e14, M, P)
Pb = np.interp(2.0e15, M, P)

In [ ]:
uu = prng.uniform(low=Pa, high=Pb, size=num_clusters)
M200c = np.interp(uu, P, M, left=0.0, right=1.0)

In [ ]:
r200c = (3*M200c/(4.0*np.pi*(200.0*rho_crit)))**(1./3.)

In [ ]:
b = -0.101
a = 0.905
logc = a+b*np.log10(M200c/(1.0e12/h))
c200c = 10**logc

In [ ]:
sigma = 0.11/np.log10(np.exp(1.0)) # dex to natural log
c200s = prng.lognormal(mean=np.log(c200c), sigma=sigma)

In [ ]:
r_s = r200c/c200s

In [ ]:
g = lambda c: 1.0/(np.log(1.0+c)-c/(1.0+c))

In [ ]:
rho_s = 200.0*rho_crit*c200s**3*g(c200s)/3.

In [ ]:
radii = np.linspace(0.0, 2000.0, 4000)

In [ ]:
rho_nfw = lambda r, rho_s, r_s: rho_s/((r/r_s)*(1.0+r/r_s)**2)
m_nfw = lambda r, rho_s, r_s: 4.0*np.pi*rho_s*r_s**3*(np.log(1.0+r/r_s)-r/r_s/(1.0+r/r_s))

In [ ]:
rho_tot = []
m_tot = []
m_b = []
rho_b = []
m_pred = []
for i in range(num_clusters):
    rho_tot.append(rho_nfw(radii, rho_s[i], r_s[i]))
    Mtot = lambda r: m_nfw(r, rho_s[i], r_s[i])
    def dMbdr(r, Mb): 
        r = max(r, 1.0e-4)
        return -Mb/r + K*(Mtot(r)-Mb)**2/r**3
    m_tot.append(Mtot(radii))
    sol = solve_ivp(dMbdr, (radii[0], radii[-1]), [0.0], t_eval=radii)
    m_b.append(sol.y[0,:])
    frho = InterpolatedUnivariateSpline(radii, m_b[i])
    rho_b.append(frho(radii, nu=1)/(4.0*np.pi*radii*radii))
rho_tot = np.array(rho_tot)
m_tot = np.array(m_tot)
m_b = np.array(m_b)
rho_b = np.array(rho_b)
m_pred = m_b+np.sqrt(radii**2*(m_b+4.0*np.pi*radii**3*rho_b)/K)

In [ ]:
rmax = np.array([radii[rho > 0][-1] for rho in rho_b])

In [ ]:
mue = 1.0/0.88
n_e = (rho_b*u.Msun/u.kpc**3).to_equivalent("cm**-3", "number_density", mu=mue)

In [ ]:
rho_avg = 3.0*m_tot/(4.0*np.pi*radii**3)

In [ ]:
r500c = []
for i in range(num_clusters):
    r500c.append(radii[rho_avg[i,:] < 500.0*rho_crit][0])
r500c = np.array(r500c)
M500c = (4.0*np.pi/3.0)*500.0*rho_crit*r500c**3

In [ ]:
# Find stellar contribution to baryon mass using Lin et al 2004 and Gonzalez et al 2013
c_s = 2.9
rho_ss = 1.0
A = 0.39*(M500c*1.0e-14)**(-0.84)
m_star = []
rho_star = []
for i in range(num_clusters):
    r_ss = r200c[i]/c_s 
    m_s = m_nfw(radii, rho_ss, r_ss)
    idx = np.searchsorted(radii, r500c[i])-1
    m_s /= m_s[idx]
    m_s *= (A[i]/(1.0+A[i]))*m_b[i,idx]
    frho = InterpolatedUnivariateSpline(radii, m_s)
    rho_s = frho(radii, nu=1)/(4.0*np.pi*radii*radii)
    m_star.append(m_s)
    rho_star.append(rho_s)
m_star = np.array(m_star)
rho_star = np.array(rho_star)
m_gas = m_b-m_star
rho_gas = rho_b-rho_star
f_gas = m_gas/m_tot

In [ ]:
#r_core = 0.012*r500c

In [ ]:
# Put this step in a loop and replace 0 by i in the loop
#y = 4.0*np.pi*radii**2*n_e[0,:].d
#V = 4.0*np.pi/3.0*r_core[0]**3
#which_radii = np.logical_and(0.0 < radii, radii <= r_core[0])
#n_e_avg = np.trapz(y[which_radii], radii[which_radii])/V

In [ ]:
m_b.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.loglog(radii, rho_tot[100])
plt.loglog(radii, rho_b[100])
plt.loglog(radii, rho_gas[100])
plt.loglog(radii, rho_star[100])